# **ResNet with MNIST**

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from sklearn.model_selection import train_test_split

# ========== Load & Preprocess MNIST ==========
def load_mnist_images(file_path):
    with open(file_path, "rb") as f:
        _ = int.from_bytes(f.read(4), byteorder="big")
        num_images = int.from_bytes(f.read(4), byteorder="big")
        rows = int.from_bytes(f.read(4), byteorder="big")
        cols = int.from_bytes(f.read(4), byteorder="big")
        data = f.read()
    images = np.frombuffer(data, dtype=np.uint8).reshape((num_images, rows, cols, 1))
    return images.astype("float32") / 255.0

X = load_mnist_images("/content/drive/MyDrive/Colab Notebooks/Major-project-8th-sem/mnist.idx3-ubyte.csv")
y = np.random.randint(0, 2, size=(X.shape[0], 1)).astype("float32")

# Reduce to 10k for speed/safety (optional)
X = X[:10000]
y = y[:10000]

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ========== tf.data.Dataset with On-the-fly Resize ==========
AUTOTUNE = tf.data.AUTOTUNE
BATCH_SIZE = 32

def preprocess(x, y):
    x = tf.image.resize(x, [224, 224])
    x = tf.image.grayscale_to_rgb(x)
    return x, y

train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).map(preprocess).batch(BATCH_SIZE).prefetch(AUTOTUNE)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).map(preprocess).batch(BATCH_SIZE).prefetch(AUTOTUNE)

# ========== Custom Metrics ==========
L = 73

def ours_accu(y_true, y_pred):
    y_pred = 1 / (1 + tf.math.exp(-L * (y_pred - 0.5)))
    accu = (tf.cast(tf.shape(y_true)[0], tf.float32) - tf.reduce_sum(y_true) - tf.reduce_sum(y_pred) + 2 * tf.reduce_sum(y_true * y_pred)) / tf.cast(tf.shape(y_true)[0], tf.float32)
    return 1 - accu

def ours_fbeta(y_true, y_pred, beta=1):
    y_pred = 1 / (1 + tf.math.exp(-L * (y_pred - 0.5)))
    numerator = (1 + beta**2) * tf.reduce_sum(y_true * y_pred)
    denominator = (beta**2) * tf.reduce_sum(y_true) + tf.reduce_sum(y_pred)
    return 1 - (numerator / (denominator + 1e-8))

def ours_gmean(y_true, y_pred):
    y_pred = 1 / (1 + tf.math.exp(-L * (y_pred - 0.5)))
    syhy = tf.reduce_sum(y_true * y_pred)
    sy = tf.reduce_sum(y_true)
    yl = tf.cast(tf.shape(y_true)[0], tf.float32)
    gmean = tf.sqrt(syhy * (yl - tf.reduce_sum(y_pred) - sy + syhy) / (sy * (yl - sy) + 1e-8))
    return 1 - gmean

def ours_baccu(y_true, y_pred):
    y_pred = 1 / (1 + tf.math.exp(-L * (y_pred - 0.5)))
    syhy = tf.reduce_sum(y_true * y_pred)
    sy = tf.reduce_sum(y_true)
    yl = tf.cast(tf.shape(y_true)[0], tf.float32)
    baccu = (yl * (syhy + sy) - sy * (tf.reduce_sum(y_pred) + sy)) / (2 * sy * (yl - sy) + 1e-8)
    return 1 - baccu

# ========== Build ResNet50 Model ==========
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))
base_model.trainable = False  # Freeze base

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(64, activation='relu')(x)
output = Dense(1)(x)  # Output logits

model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss='mse', metrics=['accuracy'])

# ========== Train ==========
model.fit(train_ds, epochs=2, validation_data=test_ds)

# ========== Predict & Evaluate ==========
# Get all test data for metric computation
X_test_resized = []
for x, _ in test_ds:
    X_test_resized.append(x)
X_test_resized = tf.concat(X_test_resized, axis=0)

y_test_full = tf.concat([y for _, y in test_ds], axis=0)
y_pred_raw = model.predict(X_test_resized)

print(f"Custom Accuracy      : {ours_accu(y_test_full, y_pred_raw).numpy():.4f}")
print(f"Custom F-beta        : {ours_fbeta(y_test_full, y_pred_raw).numpy():.4f}")
print(f"Custom Geometric Mean: {ours_gmean(y_test_full, y_pred_raw).numpy():.4f}")
print(f"Custom Balanced Acc. : {ours_baccu(y_test_full, y_pred_raw).numpy():.4f}")


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/2


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor']
Received: inputs=Tensor(shape=(None, 224, 224, 3))
  warnings.warn(msg)


250/250 ━━━━━━━━━━━━━━━━━━━━ 1726s 7s/step - accuracy: 0.5127 - loss: 0.2967 - val_accuracy: 0.5080 - val_loss: 0.2564
Epoch 2/2
250/250 ━━━━━━━━━━━━━━━━━━━━ 1740s 7s/step - accuracy: 0.5114 - loss: 0.2663 - val_accuracy: 0.5075 - val_loss: 0.2558


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor']
Received: inputs=Tensor(shape=(32, 224, 224, 3))
  warnings.warn(msg)


63/63 ━━━━━━━━━━━━━━━━━━━━ 327s 5s/step
Custom Accuracy      : 0.4928
Custom F-beta        : 0.9845
Custom Geometric Mean: 0.9117
Custom Balanced Acc. : 0.5002
